In [2]:
import numpy as np
import pandas as pd
import time
import math
from scipy import signal

from datetime import datetime
import csv

# import matplotlib.pyplot as plt
# from pyedflib import highlevel       # pip install pyEDFlib


In [3]:
# Read the sleep stages (1/30 Hz)
file = 'data/20220621_Ike/stage.csv'
file_stage = pd.read_csv(file)

def stage_value(x):
    if x=='N1':
        return 1
    elif x=='N2':
        return 1
    elif x=='N3':
        return 1
    else:
        return 0

STAGE = file_stage['Sleep Stage'].apply(stage_value)
STAGE = np.array(STAGE)

f = open('220621_Ike_STAGE.csv', 'w')
writer = csv.writer(f)
writer.writerow(STAGE)
f.close()

# data start time
st = file_stage['Time [hh:mm:ss.xxx]'][0]
st = datetime.strptime(st[:19], '%Y-%m-%dT%H:%M:%S')
start_in_sec = (st.hour*60 + st.minute)*60 + st.second


In [4]:
# Read the PPG data
file = 'data/20220621_Ike/220621_Ike_PPG.csv'
file_PPG = pd.read_csv(file, header=None)

ppgFs = 20
DownFs = 10
R = np.array(file_PPG.iloc[:, 1])
IR = np.array(file_PPG.iloc[:, 2])
G = np.array(file_PPG.iloc[:, 3])
total_sec = np.floor(len(R)/ppgFs).astype(int)
print(total_sec)

file = 'data/20220621_Ike/stage.csv'
file_stage = pd.read_csv(file)

st = file_stage['Time [hh:mm:ss.xxx]'][0]
st = datetime.strptime(st[:19], '%Y-%m-%dT%H:%M:%S')
start_in_sec = (st.hour*60 + st.minute)*60 + st.second


timestr = file_PPG.iloc[:, 7].astype(str)
ppgst = datetime.strptime(timestr[0][:-3], '%Y%m%d%H%M%S')
ppgst = (ppgst.hour*60 + ppgst.minute)*60 + ppgst.second


ppgFs = 20
DownFs = 10
R = np.array(file_PPG.iloc[20*(start_in_sec-ppgst):, 1])
IR = np.array(file_PPG.iloc[20*(start_in_sec-ppgst):, 2])
G = np.array(file_PPG.iloc[20*(start_in_sec-ppgst):, 3])
# total_sec = np.floor(len(R)/ppgFs).astype(int)
# print(total_sec)

df = pd.DataFrame({'R': R,
                   'IR': IR,
                   'G': G})
df.to_csv('220621_Ike_ppg.csv',index=False,header=False)


23947


In [11]:
# Read the apnea events (2 Hz)
file = 'data/20220621_Ike/event.csv'
file_apnea = pd.read_csv(file)
file = 'data/20220621_Ike/220621_Ike_PPG.csv'
file_PPG = pd.read_csv(file, header=None)
file = 'data/20220621_Ike/stage.csv'
file_stage = pd.read_csv(file)

ppgFs = 20

R = np.array(file_PPG.iloc[:, 1])
print(R)
total_sec = np.floor(len(R)/ppgFs).astype(int)
print(total_sec)



st = file_stage['Time [hh:mm:ss.xxx]'][0]
st = datetime.strptime(st[:19], '%Y-%m-%dT%H:%M:%S')
print(st)
start_in_sec = (st.hour*60 + st.minute)*60 + st.second


CSA = 1
OSA = 2
MIX = 2
HYP = 3

apnea = np.zeros(total_sec*2)

for i in range(len(file_apnea)):
    x = file_apnea['Event'][i]
    tm = datetime.strptime(file_stage['Time [hh:mm:ss.xxx]'][i][:19], '%Y-%m-%dT%H:%M:%S')
    print(tm)
    hh = tm.hour
    if hh<12:
        hh = hh+24
    mm = tm.minute
    ss = tm.second
    if x=='Central apnea' or x=='Hypopnea Central':
        dura = int(2*file_apnea['Duration[s]'][i])
        Time_in_sec = (hh*60 + mm)*60 + ss
        if (Time_in_sec - start_in_sec) < 0:
            print('Error time.')
            print(severity + ' - ' + key + ' at row ' + str(i+1))
        event_start = max(0, Time_in_sec - start_in_sec) * 2
        event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
        apnea[event_start : event_end] = CSA
        
    elif x=='Obstructive apnea' or x=='Hypopnea Obstructive':
        dura = int(2*file_apnea['Duration[s]'][i])
        Time_in_sec = (hh*60 + mm)*60 + ss
        if (Time_in_sec - start_in_sec) < 0:
            print('Error time.')
            print(severity + ' - ' + key + ' at row ' + str(i+1))
        event_start = max(0, Time_in_sec - start_in_sec) * 2
        event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
        apnea[event_start : event_end] = OSA
        
    elif x=='Mixed apnea' or x=='Hypopnea Mixed':
        dura = int(2*file_apnea['Duration[s]'][i])
        Time_in_sec = (hh*60 + mm)*60 + ss
        if (Time_in_sec - start_in_sec) < 0:
            print('Error time.')
            print(severity + ' - ' + key + ' at row ' + str(i+1))
        event_start = max(0, Time_in_sec - start_in_sec) * 2
        event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
        apnea[event_start : event_end] = MIX
        
    elif x=='Hypopnea':
        dura = int(2*file_apnea['Duration[s]'][i])
        print(dura)
        
        Time_in_sec = (hh*60 + mm)*60 + ss
        print(Time_in_sec)
        if (Time_in_sec - start_in_sec) < 0:
            print('Error time.')
            print(severity + ' - ' + key + ' at row ' + str(i+1))
        event_start = max(0, Time_in_sec - start_in_sec) * 2
        event_end   = max(0, ((Time_in_sec - start_in_sec) * 2 + dura))
        apnea[event_start : event_end] = HYP

        
f = open('220621_Ike_apnea.csv', 'w')
writer = csv.writer(f)
writer.writerow(apnea)
f.close()

[659845 659572 659716 ... 604018 604050 603916]
23947
2022-06-21 22:12:41
2022-06-21 22:12:41
2022-06-21 22:13:11
2022-06-21 22:13:41
2022-06-21 22:14:11
2022-06-21 22:14:41
2022-06-21 22:15:11
2022-06-21 22:15:41
2022-06-21 22:16:11
2022-06-21 22:16:41
2022-06-21 22:17:11
2022-06-21 22:17:41
2022-06-21 22:18:11
2022-06-21 22:18:41
47
80321
2022-06-21 22:19:11
2022-06-21 22:19:41
2022-06-21 22:20:11
2022-06-21 22:20:41
2022-06-21 22:21:11
2022-06-21 22:21:41
2022-06-21 22:22:11
2022-06-21 22:22:41
2022-06-21 22:23:11
2022-06-21 22:23:41
2022-06-21 22:24:11
64
80651
2022-06-21 22:24:41
2022-06-21 22:25:11
2022-06-21 22:25:41
129
80741
2022-06-21 22:26:11
2022-06-21 22:26:41
38
80801
2022-06-21 22:27:11
2022-06-21 22:27:41
40
80861
2022-06-21 22:28:11
151
80891
2022-06-21 22:28:41
2022-06-21 22:29:11
2022-06-21 22:29:41
2022-06-21 22:30:11
2022-06-21 22:30:41
2022-06-21 22:31:11
2022-06-21 22:31:41
2022-06-21 22:32:11
2022-06-21 22:32:41
2022-06-21 22:33:11
2022-06-21 22:33:41
2022-06-21

In [5]:
print(sum(apnea))

10854.0


In [6]:
# Read the sleep stages (1/30 Hz)





def stage_value(x):
    if x=='REM':
        return 12
    elif x=='N1':
        return 13
    elif x=='N2':
        return 14
    elif x=='N3':
        return 15
    else:
        return 11
def trans(x):
    with open(x) as f:
        
        # st = x['Time [hh:mm:ss.xxx]'][0]
        # st = datetime.strptime(line[:19], '%Y-%m-%dT%H:%M:%S')
        # start_in_sec = (st.hour*60 + st.minute)*60 + st.second

        SleepStage=[]
        data = f.readlines()####加入開始時間
        #print(result)
        #data = result[result.index("sleep") :]
        #print(data[19])
        a=[]
        for line in data:
            s = line.split('\t')
            SleepStage.append(s[0])
        
        SleepStage = SleepStage[SleepStage.index("Sleep Stage") + 1:]  
        for i in SleepStage:  
            STAGE=stage_value(i)   
            a.append(STAGE)
        STAGE = np.array(a)
     #print(STAGE)
    return STAGE





file = 'data/20220621_ike_raw/1110621-stage.txt'
STAGE=trans(file)
st = file_stage['Time [hh:mm:ss.xxx]'][0]
print(st)

print(type(st))
print(st[:19])
st = datetime.strptime(st[:19], '%Y-%m-%dT%H:%M:%S')


start_in_sec = (st.hour*60 + st.minute)*60 + st.second
f = open('220621_Tai_STAGE.csv', 'w')
writer = csv.writer(f)
writer.writerow(STAGE)
f.close()

2022-06-21T22:12:41.000000
<class 'str'>
2022-06-21T22:12:41
